In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

import datetime


In [23]:
df_interactions = pd.read_csv("../data/daily_news.csv", index_col='date')

In [24]:
df_interactions.head()

,buba,title,image,link,likes
date,,,,,
2023-01-02,0,Perché nel 2023 benzina e autostrade costano d...,https://media-assets.wired.it/photos/636cf8a50...,https://www.wired.it/article/2023-benzina-auto...,0
2023-01-02,1,Il vaccino mRna per l'influenza che si sommini...,https://media-assets.wired.it/photos/63b2c85b5...,https://www.wired.it/article/vaccino-mrna-infl...,0
2023-01-02,2,Come sono stati catturati i peggiori cyber cri...,https://media-assets.wired.it/photos/63b2b5ba0...,https://www.wired.it/article/peggiori-cyber-cr...,0
2023-01-02,3,Jeremy Renner travolto dalla neve: è in condiz...,https://media-assets.wired.it/photos/63b29901a...,https://www.wired.it/article/jeremy-renner-ric...,0
2023-01-02,4,Harry Potter: tutto quello che sappiamo sul re...,https://media-assets.wired.it/photos/63b2b03e5...,https://www.wired.it/article/harry-potter-rebo...,0


In [26]:
df_interactions.head()


,title,image,link,likes
date,,,,
2023-01-02,Perché nel 2023 benzina e autostrade costano d...,https://media-assets.wired.it/photos/636cf8a50...,https://www.wired.it/article/2023-benzina-auto...,0
2023-01-02,Il vaccino mRna per l'influenza che si sommini...,https://media-assets.wired.it/photos/63b2c85b5...,https://www.wired.it/article/vaccino-mrna-infl...,0
2023-01-02,Come sono stati catturati i peggiori cyber cri...,https://media-assets.wired.it/photos/63b2b5ba0...,https://www.wired.it/article/peggiori-cyber-cr...,0
2023-01-02,Jeremy Renner travolto dalla neve: è in condiz...,https://media-assets.wired.it/photos/63b29901a...,https://www.wired.it/article/jeremy-renner-ric...,0
2023-01-02,Harry Potter: tutto quello che sappiamo sul re...,https://media-assets.wired.it/photos/63b2b03e5...,https://www.wired.it/article/harry-potter-rebo...,0


In [27]:
df_interactions.to_csv('../data/daily_news.csv')

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"} 

In [3]:
URL = 'https://www.lastampa.it'

In [4]:
page = requests.get(URL, headers=headers)

In [5]:
# Parse the HTML content
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
# Seleziona l'elemento
articles = soup.select("article[class^='entry default']")

In [13]:
title = articles[1].select_one('h2').text.strip()
print(title)

Accoltellata alla stazione Termini, l’ipotesi: colpita da uno sbandato perché non gli ha dato dei soldi


In [14]:
link = articles[1].select_one('a')['href']
print(link)

https://www.lastampa.it/cronaca/2023/01/02/news/le_coltellate_i_blitz_il_movente_nelle_riprese_video_la_verita_sullaccoltellamento_di_termini-12443967/


In [26]:
print(articles[0])

<article class="entry default" data-sindex="1">
<figure class="entry__media">
<a href="https://www.lastampa.it/cronaca/2023/01/02/news/zaia_intercettato_sulla_guerra_dei_tamponi_rapidi_crisanti_lo_faro_schiantare-12443591/">
<picture>
<source height="220" media="(max-width: 767px)" srcset="https://www.gedistatic.it/content/gnn/img/lastampa/2023/01/02/185413663-bfa8625d-39cf-4826-adeb-6e636067e787.jpg?webp" type="image/webp" width="390"/>
<source height="720" srcset="https://www.gedistatic.it/content/gnn/img/lastampa/2023/01/02/185413659-777fdcb7-4e50-44c6-abbe-0bc8eb01053c.jpg?webp" type="image/webp" width="1280"/>
<source height="220" media="(max-width: 767px)" srcset="https://www.gedistatic.it/content/gnn/img/lastampa/2023/01/02/185413663-bfa8625d-39cf-4826-adeb-6e636067e787.jpg" width="390"/>
<img height="720" src="https://www.gedistatic.it/content/gnn/img/lastampa/2023/01/02/185413659-777fdcb7-4e50-44c6-abbe-0bc8eb01053c.jpg" width="1280"/>
</picture>
</a>
</figure>
<div class="ent

In [35]:
for i in range(len(articles)):
    try:
       imagine = articles[i].select_one("source")['data-srcset']
    except:
        pass


In [43]:
# Crea una lista vuota che conterrà i dati estratti
data = []

# Itera su ogni elemento
for i in range(len(articles)):
    # Estrai il titolo
    title = articles[i].select_one('h2').text.strip()

    # Estrai il link
    link = articles[i].select_one('a')['href']

    #estrai immagine
    try:
       img = articles[i].select_one("source")['data-srcset']
    except:
        pass

    # Aggiungi i dati estratti alla lista
    data.append({
        'title': title,
        'image': img,
        'link': link
    })

# Crea un DataFrame Pandas a partire dalla lista di dati
df = pd.DataFrame(data)

In [44]:
df['date'] = datetime.datetime.today().strftime("%Y-%m-%d")

In [45]:
df.set_index('date', inplace=True)

,title,image,link
date,,,
2023-01-02,"Zaia, Crisanti e la guerra delle intercettazio...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/cronaca/2023/01/02/new...
2023-01-02,"Accoltellata alla stazione Termini, l’ipotesi:...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/cronaca/2023/01/02/new...
2023-01-02,La corsa della benzina dopo la fine degli scon...,https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/economia/2023/01/02/ne...
2023-01-02,Qatargate: via l’immunità per Cozzolino e Tara...,https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/esteri/2023/01/02/news...
2023-01-02,"Guerra Russia Ucraina, droni su Kiev, black ou...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/esteri/2023/01/02/dire...


In [46]:
df.tail()

,title,image,link
date,,,
2023-01-02,"Da Glory a Matilde, la carica dei nuovi nati: ...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/02/news...
2023-01-02,"2023, l’anno del Liberty a Torino: a Palazzo M...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/02/news...
2023-01-02,"“2022, un anno da record per il Teatro Stabile...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/02/news...
2023-01-02,Della Casa: “Torneremo a riempire i cinema se ...,https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/02/news...
2023-01-02,Ecco la selezione delle migliori offerte della...,https://www.gedistatic.it/content/gnn/img/last...,https://www.consigli.it/migliori-offerte-del-g...


In [42]:
df.to_csv('data/daily_news.csv', mode='a', header=False)